In [251]:
import os, sys, datetime
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from sklearn.datasets import load_iris
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import classification_report, confusion_matrix

In [252]:
dataframe = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

dataframe.head()

#dataframe.gender.unique()
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [253]:
# Affiche les valeurs non convertibles en float
print(dataframe[~dataframe["TotalCharges"].str.replace('.', '', 1).str.isnumeric()]["TotalCharges"].unique())

[' ']


In [254]:
# Remplacer les chaînes vides (ou contenant uniquement des espaces) par NaN
dataframe["TotalCharges"] = dataframe["TotalCharges"].replace(" ", np.nan)

# Convertir en float
dataframe["TotalCharges"] = dataframe["TotalCharges"].astype(float)

In [255]:
# Affiche le nombre de NaN par colonne
print(dataframe.isna().sum())


customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64


In [256]:
# Colonnes avec au moins un NaN
cols_with_nan = dataframe.columns[dataframe.isna().any()].tolist()
print("Colonnes avec des valeurs manquantes :", cols_with_nan)


Colonnes avec des valeurs manquantes : ['TotalCharges']


In [257]:
dataframe.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [258]:
dataframe.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [259]:
#dataframe = dataframe.dropna(subset=["TotalCharges", "MonthlyCharges"])


In [260]:
# Trouver les colonnes contenant uniquement les valeurs 'Yes' et 'No'
binary_cols = [
    col for col in dataframe.columns
    if set(dataframe[col].dropna().unique()) <= {'Yes', 'No'}
]

print("Colonnes binaires (Yes/No) :", binary_cols)

Colonnes binaires (Yes/No) : ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']


In [261]:
one_hot_cols = [
    col for col in dataframe.select_dtypes(include='object').columns
    if dataframe[col].nunique(dropna=True) > 2
]

print("Colonnes à encoder en One-Hot :", one_hot_cols)

Colonnes à encoder en One-Hot : ['customerID', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']


In [262]:
print(dataframe.columns)


Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


In [ ]:
# a) Colonnes binaires Yes/No
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

y = dataframe['Churn'].map({"Yes": 1, "No": 0}) 
dataframe = dataframe.drop(columns=["Churn", "customerID"])


binary_cols = [
    c for c in dataframe.columns
    if set(dataframe[c].dropna().unique()) <= {"Yes", "No"}
]

print("Colonnes à encoder en OrdinalEncoder :", binary_cols)

Colonnes à encoder en OrdinalEncoder : ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']


In [264]:
# b) Colonnes catégorielles à > 2 modalités
multi_cat_cols = [
    c for c in dataframe.select_dtypes(include="object").columns
    if c not in binary_cols
]

print("Colonnes à encoder en One-Hot :", multi_cat_cols)

Colonnes à encoder en One-Hot : ['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']


In [265]:
# c) Colonnes numériques
numeric_cols = dataframe.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Colonnes à encoder en StandardScaler :", numeric_cols)

Colonnes à encoder en StandardScaler : ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']


In [266]:
#dataframe.head()

In [267]:
X_train, X_test, y_train, y_test = train_test_split(dataframe, y, stratify=y, test_size=0.2, random_state=42)
X_train.shape
#X_test.shape

(5634, 19)

In [268]:
# ------------------------------------------------------------------
# 3. Pipelines de pré-traitement
# ------------------------------------------------------------------
binary_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder(categories=[["No", "Yes"]]*len(binary_cols)))
])

# Pipeline pour les colonnes multi-catégorielles (>2 modalités)
multi_cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
# Optional : colonnes numériques
numeric_cols = dataframe.select_dtypes(include=["int64", "float64"]).columns.tolist()

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# ------------------------------------------------------------------
# 4. Assemblage + modèle final
# ------------------------------------------------------------------
preprocessor = ColumnTransformer([
    ("bin", binary_pipeline, binary_cols),
    ("cat", multi_cat_pipeline, multi_cat_cols),
    ("num", numeric_pipeline, numeric_cols)
])

# Entraîner le pipeline sur les données
preprocessor.fit(X_train)

# Transformation
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

print(X_train_preprocessed[:5])
print(X_test_preprocessed[:5])

"""# Il faiy faire attention, appliquer les transformations sur les numériques (base train, valid et test)


clf = Pipeline([
    ("prep",  preprocessor),
    ("model", LogisticRegression(max_iter=1000, n_jobs=-1))
])

clf.fit(dataframe, y)                # ↖️ pipeline prêt à l’emploi

X_transformed = clf.named_steps["prep"].transform(dataframe)
print(X_transformed[:5])"""
"""""""""
# Exemple de prédiction :
proba = clf.predict_proba(dataframe.iloc[[0]])[0, 1]
print(f"Probabilité de churn pour la 1ʳᵉ ligne : {proba:.2%}")"""

[[ 0.          0.          0.          0.          0.          1.
   0.          1.          0.          1.          0.          0.
   1.          0.          0.          1.          0.          0.
   0.          0.          1.          1.          0.          0.
   0.          0.          1.          0.          0.          1.
   1.          0.          0.          0.          0.          1.
   0.         -0.44177295  0.10237124 -0.52197565 -0.26328951]
 [ 1.          1.          1.          0.          0.          1.
   1.          0.          0.          0.          1.          0.
   0.          0.          1.          1.          0.          0.
   1.          0.          0.          1.          0.          0.
   1.          0.          0.          1.          0.          0.
   1.          0.          0.          0.          0.          0.
   1.         -0.44177295 -0.71174346  0.33747781 -0.50481497]
 [ 1.          1.          0.          0.          0.          1.
   0.          1

'\n# Exemple de prédiction :\nproba = clf.predict_proba(dataframe.iloc[[0]])[0, 1]\nprint(f"Probabilité de churn pour la 1ʳᵉ ligne : {proba:.2%}")'

In [269]:
X_train_preprocessed.shape

(5634, 41)